In [1]:
import csv
import os
import time
import ujson
from random import randint
from typing import Dict, List, Any

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager

In [148]:
def crawlDetail(driver, uri):
    driver.get(uri)
    abstract = driver.find_element(
        By.CSS_SELECTOR,"div[class='content-content publication-content']>div"
    )
    abs_text= abstract.text if abstract is not None else ""
    uri = uri + "/fingerprints/"
    driver.get(uri)
    finger_print = driver.find_elements(
        By.CSS_SELECTOR,"div[class='publication-fingerprint-thesauri']>h3"
    )
    finger_text = [x.text for x in finger_print] if finger_print is not None else []
    return [
        abs_text,
        finger_text
    ]

In [149]:
def write_authors(list1, file_name):
    with open(file_name, 'w', encoding='utf-8') as f:
        for i in range(0, len(list1)):
            f.write(list1[i] + '\n')

In [164]:
pub_data = []

In [166]:
def threadRipper(links):
    global pub_data
    options = webdriver.ChromeOptions()
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    options.add_argument('--headless')
    
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    
    for link in links:
        print(link)
        # Visit each link to get data
        result = driver.get(link)
        driver.get(link)
        try:
            name = driver.find_element(By.CSS_SELECTOR,"div[class='header person-details']>h1").text
            if driver.find_elements(By.CSS_SELECTOR,".portal_link.btn-primary.btn-large"):
                element = driver.find_elements(By.CSS_SELECTOR,".portal_link.btn-primary.btn-large")
                for a in element:
                    if "research output".lower() in a.text.lower():
                        driver.execute_script("arguments[0].click();", a)
                        driver.get(driver.current_url)
                        r = requests.get(driver.current_url)
                        soup = BeautifulSoup(r.content, 'lxml')
                        table = soup.find('ul', attrs={'class': 'list-results'})
                        if table != None:
                            for row in table.findAll('div', attrs={'class': 'result-container'}):
                                data = {}
                                data['name'] = row.h3.a.text
                                data['pub_url'] = row.h3.a['href']
                                date = row.find("span", class_="date")

                                rowitem = row.find_all(['div'])
                                span = row.find_all(['span'])
                                data['cu_author'] = name
                                data['date'] = date.text
                                
                                l3 = crawlDetail(driver, data['pub_url'])
                                data["abstract"] = l3[0]
                                data["category"] = l3[1]
                                pub_data.append(data)
            else:
                r = requests.get(link)
                soup = BeautifulSoup(r.content, 'lxml')
                table = soup.find('div', attrs={'class': 'relation-list relation-list-publications'})
                if table != None:
                    for row in table.findAll('div', attrs={'class': 'result-container'}):
                        data = {}
                        data["name"] = row.h3.a.text
                        data['pub_url'] = row.h3.a['href']
                        date = row.find("span", class_="date")
                        rowitem = row.find_all(['div'])
                        span = row.find_all(['span'])
                        data['cu_author'] = name
                        data['date'] = date.text
                        
                        l3 = crawlDetail(driver, data['pub_url'])
                        data["abstract"] = l3[0]
                        data["category"] = l3[1]
                        pub_data.append(data)
        except Exception as e:
            print(e)
            continue
    driver.quit()

In [167]:
import threading
def initCrawlerScraper(uri):
    global pub_data
    # Initialize driver for Chrome
    options = webdriver.ChromeOptions()
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    options.add_argument('--headless')
    
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    driver.get(uri)
    
    Links = [] 
    nextLink = driver.find_element(By.CSS_SELECTOR,".nextLink").is_enabled()
    while (nextLink):
        page = driver.page_source
        bs = BeautifulSoup(page, "lxml")

        for link in bs.findAll('a', class_='link person'):
            url = str(link)[str(link).find('https://pureportal.coventry.ac.uk/en/persons/'):].split('"')
            Links.append(url[0])
        try:
            if driver.find_element(By.CSS_SELECTOR,".nextLink"):
                element = driver.find_element(By.CSS_SELECTOR,".nextLink")
                driver.execute_script("arguments[0].click();", element)
            else:
                nextLink = False
        except NoSuchElementException:
            break
    write_authors(Links, 'Authors_URL.txt')
    Links = Links
    chunk_size = len(Links) // 25  # Calculate the size of each chunk
    chunks = [Links[i:i+chunk_size] for i in range(0, len(Links), chunk_size)]

    # If there are remaining items, add them to the last chunk
    if len(Links) % chunk_size != 0:
        chunks[-1].extend(Links[-(len(Links) % chunk_size):])
    try:
        _threader = []
        for thread in chunks:
            t = threading.Thread(target=threadRipper, args=(thread,))
            _threader.append(t)
            t.start()
        for t in _threader:
            t.join()
    except:
        pass

    print("Crawler has scrapped data for ", len(pub_data), " pureportal publications")
    driver.quit()

    with open('res.json', 'w') as f:
        ujson.dump(pub_data, f)

In [168]:
pub_data = []
initCrawlerScraper('https://pureportal.coventry.ac.uk/en/organisations/coventry-university/persons/')

/tmp/ipykernel_261630/219374300.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
/tmp/ipykernel_261630/138250350.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


https://pureportal.coventry.ac.uk/en/persons/emma-calcutt
https://pureportal.coventry.ac.uk/en/persons/nina-atkinson
https://pureportal.coventry.ac.uk/en/persons/aliano-abbasi
https://pureportal.coventry.ac.uk/en/persons/shenal-rajintha-alexander-samarathunge-gunawardena
https://pureportal.coventry.ac.uk/en/persons/scott-delahunta
https://pureportal.coventry.ac.uk/en/persons/jane-coad
https://pureportal.coventry.ac.uk/en/persons/rachel-bevins
https://pureportal.coventry.ac.uk/en/persons/lamis-jamil
https://pureportal.coventry.ac.uk/en/persons/ibrahim-alkhataybeh
https://pureportal.coventry.ac.uk/en/persons/ikenna-stanley-enebuse
https://pureportal.coventry.ac.uk/en/persons/ahmed-nawaz-khan
https://pureportal.coventry.ac.uk/en/persons/michael-hills
https://pureportal.coventry.ac.uk/en/persons/francesca-mantia-conaty
https://pureportal.coventry.ac.uk/en/persons/beth-gardiner
https://pureportal.coventry.ac.uk/en/persons/seyed-mohamad-javad-sajjadi-shourije
https://pureportal.coventry.ac.u

https://pureportal.coventry.ac.uk/en/persons/mehtap-hisarciklilar
https://pureportal.coventry.ac.uk/en/persons/john-allen
https://pureportal.coventry.ac.uk/en/persons/mirabel-pelton
https://pureportal.coventry.ac.uk/en/persons/rohit-bhagat
https://pureportal.coventry.ac.uk/en/persons/ami-widyastuti
https://pureportal.coventry.ac.uk/en/persons/julia-wieczorek
https://pureportal.coventry.ac.uk/en/persons/poppy-sall
https://pureportal.coventry.ac.uk/en/persons/dawn-wilbraham-hemmings
https://pureportal.coventry.ac.uk/en/persons/lawrence-ndupu
https://pureportal.coventry.ac.uk/en/persons/naveed-salman
https://pureportal.coventry.ac.uk/en/persons/laura-wilde
https://pureportal.coventry.ac.uk/en/persons/catalina-neculai
https://pureportal.coventry.ac.uk/en/persons/chris-salt
https://pureportal.coventry.ac.uk/en/persons/mohamed-abdelshafy
https://pureportal.coventry.ac.uk/en/persons/tej-samani
https://pureportal.coventry.ac.uk/en/persons/dana-sammut
https://pureportal.coventry.ac.uk/en/person

Message: stale element reference: stale element not found
  (Session info: headless chrome=114.0.5735.90)
Stacktrace:
#0 0x558c7142d4e3 <unknown>
#1 0x558c7115cc76 <unknown>
#2 0x558c71169a71 <unknown>
#3 0x558c711616f3 <unknown>
#4 0x558c711601d6 <unknown>
#5 0x558c7116297e <unknown>
#6 0x558c71162a4c <unknown>
#7 0x558c71193922 <unknown>
#8 0x558c711b8012 <unknown>
#9 0x558c7118eb03 <unknown>
#10 0x558c711b81de <unknown>
#11 0x558c711d030e <unknown>
#12 0x558c711b7de3 <unknown>
#13 0x558c7118d2dd <unknown>
#14 0x558c7118e34e <unknown>
#15 0x558c713ed3e4 <unknown>
#16 0x558c713f13d7 <unknown>
#17 0x558c713fbb20 <unknown>
#18 0x558c713f2023 <unknown>
#19 0x558c713c01aa <unknown>
#20 0x558c714166b8 <unknown>
#21 0x558c71416847 <unknown>
#22 0x558c71426243 <unknown>
#23 0x7f1b90113bb5 <unknown>

https://pureportal.coventry.ac.uk/en/persons/stephen-austin
Message: stale element reference: stale element not found
  (Session info: headless chrome=114.0.5735.90)
Stacktrace:
#0 0x55e0d32ea4e3

https://pureportal.coventry.ac.uk/en/persons/mariam-khan
https://pureportal.coventry.ac.uk/en/persons/haseeb-ayaz
https://pureportal.coventry.ac.uk/en/persons/jason-etienne
https://pureportal.coventry.ac.uk/en/persons/sri-padma-sari
https://pureportal.coventry.ac.uk/en/persons/shahriar-khan
Message: stale element reference: stale element not found
  (Session info: headless chrome=114.0.5735.90)
Stacktrace:
#0 0x5563170e44e3 <unknown>
#1 0x556316e13c76 <unknown>
#2 0x556316e20a71 <unknown>
#3 0x556316e186f3 <unknown>
#4 0x556316e171d6 <unknown>
#5 0x556316e1997e <unknown>
#6 0x556316e19a4c <unknown>
#7 0x556316e4a922 <unknown>
#8 0x556316e6f012 <unknown>
#9 0x556316e45b03 <unknown>
#10 0x556316e6f1de <unknown>
#11 0x556316e8730e <unknown>
#12 0x556316e6ede3 <unknown>
#13 0x556316e442dd <unknown>
#14 0x556316e4534e <unknown>
#15 0x5563170a43e4 <unknown>
#16 0x5563170a83d7 <unknown>
#17 0x5563170b2b20 <unknown>
#18 0x5563170a9023 <unknown>
#19 0x5563170771aa <unknown>
#20 0x5563170cd6b8 <

...

https://pureportal.coventry.ac.uk/en/persons/sara-sharifzadeh
Message: stale element reference: stale element not found
  (Session info: headless chrome=114.0.5735.90)
Stacktrace:
#0 0x559e73ac44e3 <unknown>
#1 0x559e737f3c76 <unknown>
#2 0x559e73800a71 <unknown>
#3 0x559e737f86f3 <unknown>
#4 0x559e737f71d6 <unknown>
#5 0x559e737f997e <unknown>
#6 0x559e737f9a4c <unknown>
#7 0x559e7382a922 <unknown>
#8 0x559e7384f012 <unknown>
#9 0x559e73825b03 <unknown>
#10 0x559e7384f1de <unknown>
#11 0x559e7386730e <unknown>
#12 0x559e7384ede3 <unknown>
#13 0x559e738242dd <unknown>
#14 0x559e7382534e <unknown>
#15 0x559e73a843e4 <unknown>
#16 0x559e73a883d7 <unknown>
#17 0x559e73a92b20 <unknown>
#18 0x559e73a89023 <unknown>
#19 0x559e73a571aa <unknown>
#20 0x559e73aad6b8 <unknown>
#21 0x559e73aad847 <unknown>
#22 0x559e73abd243 <unknown>
#23 0x7fae38167bb5 <unknown>

https://pureportal.coventry.ac.uk/en/persons/mohamed-mostafa-kamel-ahmed-dawoud
https://pureportal.coventry.ac.uk/en/persons/davood-s

https://pureportal.coventry.ac.uk/en/persons/deanne-clouder
https://pureportal.coventry.ac.uk/en/persons/riann-sherratt-3
https://pureportal.coventry.ac.uk/en/persons/abdussalam-shibani
https://pureportal.coventry.ac.uk/en/persons/abbas-fotouhi
https://pureportal.coventry.ac.uk/en/persons/rebecca-fowler
https://pureportal.coventry.ac.uk/en/persons/iniobong-enang
Message: stale element reference: stale element not found
  (Session info: headless chrome=114.0.5735.90)
Stacktrace:
#0 0x558ef76be4e3 <unknown>
#1 0x558ef73edc76 <unknown>
#2 0x558ef73faa71 <unknown>
#3 0x558ef73f26f3 <unknown>
#4 0x558ef73f11d6 <unknown>
#5 0x558ef73f397e <unknown>
#6 0x558ef73f3a4c <unknown>
#7 0x558ef7424922 <unknown>
#8 0x558ef7449012 <unknown>
#9 0x558ef741fb03 <unknown>
#10 0x558ef74491de <unknown>
#11 0x558ef746130e <unknown>
#12 0x558ef7448de3 <unknown>
#13 0x558ef741e2dd <unknown>
#14 0x558ef741f34e <unknown>
#15 0x558ef767e3e4 <unknown>
#16 0x558ef76823d7 <unknown>
#17 0x558ef768cb20 <unknown>
#18 0

https://pureportal.coventry.ac.uk/en/persons/marcus-maloney
https://pureportal.coventry.ac.uk/en/persons/ania-sadkowska
https://pureportal.coventry.ac.uk/en/persons/michael-udin
https://pureportal.coventry.ac.uk/en/persons/obinna-ugwu-2
https://pureportal.coventry.ac.uk/en/persons/osita-nnanna-ugwueze
https://pureportal.coventry.ac.uk/en/persons/chioma-ukachukwu
https://pureportal.coventry.ac.uk/en/persons/arun-ulahannan
https://pureportal.coventry.ac.uk/en/persons/julio-ramirez-negrete
https://pureportal.coventry.ac.uk/en/persons/amanda-r-a-adegboye
https://pureportal.coventry.ac.uk/en/persons/eun-kyoung-lee
Message: stale element reference: stale element not found
  (Session info: headless chrome=114.0.5735.90)
Stacktrace:
#0 0x55e6f55a54e3 <unknown>
#1 0x55e6f52d4c76 <unknown>
#2 0x55e6f52e1a71 <unknown>
#3 0x55e6f52d96f3 <unknown>
#4 0x55e6f52d81d6 <unknown>
#5 0x55e6f52da97e <unknown>
#6 0x55e6f52daa4c <unknown>
#7 0x55e6f530b922 <unknown>
#8 0x55e6f5330012 <unknown>
#9 0x55e6f530

https://pureportal.coventry.ac.uk/en/persons/kumar-upvan
https://pureportal.coventry.ac.uk/en/persons/daniel-best
https://pureportal.coventry.ac.uk/en/persons/nushra-mansuri
https://pureportal.coventry.ac.uk/en/persons/ahmed-usman
https://pureportal.coventry.ac.uk/en/persons/katerina-beta
https://pureportal.coventry.ac.uk/en/persons/zrinka-uzbinec
https://pureportal.coventry.ac.uk/en/persons/noura-bettaieb
https://pureportal.coventry.ac.uk/en/persons/graham-vale-2
https://pureportal.coventry.ac.uk/en/persons/gary-betts
https://pureportal.coventry.ac.uk/en/persons/mel-vincent-dela-cruz
Message: stale element reference: stale element not found
  (Session info: headless chrome=114.0.5735.90)
Stacktrace:
#0 0x56286ad7a4e3 <unknown>
#1 0x56286aaa9c76 <unknown>
#2 0x56286aab6a71 <unknown>
#3 0x56286aaae6f3 <unknown>
#4 0x56286aaad1d6 <unknown>
#5 0x56286aaaf97e <unknown>
#6 0x56286aaafa4c <unknown>
#7 0x56286aae0922 <unknown>
#8 0x56286ab05012 <unknown>
#9 0x56286aadbb03 <unknown>
#10 0x5628

https://pureportal.coventry.ac.uk/en/persons/sandeep-gakhal
https://pureportal.coventry.ac.uk/en/persons/carolina-gallo-granizo
https://pureportal.coventry.ac.uk/en/persons/jenny-gamble
https://pureportal.coventry.ac.uk/en/persons/emily-caiger-smith
https://pureportal.coventry.ac.uk/en/persons/lorena-caires-moreira-2
Message: stale element reference: stale element not found
  (Session info: headless chrome=114.0.5735.90)
Stacktrace:
#0 0x56286ad7a4e3 <unknown>
#1 0x56286aaa9c76 <unknown>
#2 0x56286aab6a71 <unknown>
#3 0x56286aaae6f3 <unknown>
#4 0x56286aaad1d6 <unknown>
#5 0x56286aaaf97e <unknown>
#6 0x56286aaafa4c <unknown>
#7 0x56286aae0922 <unknown>
#8 0x56286ab05012 <unknown>
#9 0x56286aadbb03 <unknown>
#10 0x56286ab051de <unknown>
#11 0x56286ab1d30e <unknown>
#12 0x56286ab04de3 <unknown>
#13 0x56286aada2dd <unknown>
#14 0x56286aadb34e <unknown>
#15 0x56286ad3a3e4 <unknown>
#16 0x56286ad3e3d7 <unknown>
#17 0x56286ad48b20 <unknown>
#18 0x56286ad3f023 <unknown>
#19 0x56286ad0d1aa <un

In [169]:
with open('res.json', 'w') as f:
    ujson.dump(pub_data, f)

In [ ]:
pub_data

In [ ]:
import schedule
import time

schedule.every().monday.at("12:00").do(
    lambda: initCrawlerScraper("https://pureportal.coventry.ac.uk/en/organisations/coventry-university/persons/")
)


while True:
    schedule.run_pending()
    time.sleep(1)